In [ ]:
from datetime import datetime, timedelta
import calendar
from mrh.web.scripts.fopag_ats import fopag_ats
from mrh.web.scripts.fopag_progressao import fopag_progressao
from mrh.web.scripts.fopag_ferias_ats_prog import fopag_ferias
from mrh.web.scripts.fopag_sql import fopag_sql
import pandas as pd
import numpy as np
import re
from dotenv import load_dotenv
import os
import getpass
TETO = 37589.95


def recebe_usuario_mentorh():
    load_dotenv('.env')
    usuario=os.environ.get('MENTORH_U')
    senha=os.environ.get('MENTORH_P')
    usuario=getpass("Digite o usuário do Mentorh: ") if usuario == None else usuario
    senha=getpass("Digite o usuário do Mentorh: ") if senha == None else senha
    return usuario, senha

def encontra_data_limite(mes, ano):
    dia = calendar.monthrange(year=ano, month=mes)[1]
    data_limite = datetime(day=dia, month=mes, year=ano)
    return data_limite

def gera_ats(mes, ano):
    usuario, senha = recebe_usuario_mentorh()
    data_limite = encontra_data_limite(mes, ano)
    fopag_ats(usuario=usuario, senha=senha, data_limite=data_limite)

def gera_progressao(mes, ano):
    usuario, senha = recebe_usuario_mentorh()
    data_limite = encontra_data_limite(mes, ano)
    fopag_progressao(usuario=usuario, senha=senha, data_limite=data_limite)

def gera_relatorio_ferias(mes, ano):
    usuario, senha = recebe_usuario_mentorh()
    mes_inicio = datetime(day=1,month=mes,year=ano)
    mes_fim = encontra_data_limite(mes, ano)
    fopag_ferias(usuario=usuario, senha=senha, data_inicio=mes_inicio, data_fim=mes_fim)

def gera_relatorio_bases(mes, ano):
    usuario, senha = recebe_usuario_mentorh()
    mes_anterior = datetime(day=1,month=mes,year=ano) - timedelta(days=2)
    fopag_sql(usuario=usuario, senha=senha, destino=os.path.join(os.getcwd(), 'relatorios'), csv = True, arquivo='bases_ats_prog {}-{}'.format(mes, ano), relatorio='Base_acerto_ferias_ATS_Progressao (oficial)', argumentos_p={'p1': mes_anterior.year, 'p2': mes_anterior.month, 'p3': ano, 'p4': mes})

In [52]:
def le_ats_gerados_mentorh(mes, ano):
    df = pd.read_csv('.\\ATS - {}-{}\\ats gerados - {:02}-{}.csv'.format(mes, ano, mes, ano), names=range(12), delimiter=';')
    df_ats_mentorh = pd.DataFrame()
    df_ats_mentorh['Nome'] = [nome[:-8] for indice, nome in enumerate(df[1]) if indice%6 == 0]
    df_ats_mentorh['Matricula'] = [int(matricula[-5:]) for indice, matricula in enumerate(df[1]) if indice%6 == 0]
    adicionais_anteriores = [re.findall('\d+', adicional[-4:-2]) for indice, adicional in enumerate(df[1]) if (indice-2) >= 0 and (indice-2)%6==0]
    df_ats_mentorh['ATS anterior'] = [int(adicional[0]) if len(adicional) > 0 else 0 for adicional in adicionais_anteriores]
    df_ats_mentorh['ATS novo'] = [int(adicional[:-2]) for indice, adicional in enumerate(df[8]) if (indice-4) >= 0 and (indice-4)%6==0]
    df_ats_mentorh['Diferenca ATS'] = df_ats_mentorh['ATS novo'] - df_ats_mentorh['ATS anterior']
    return df_ats_mentorh

def pulou_mais_de_um(df):
    log = open('log.txt', 'a+')
    log.write('\n-----SERVIDORES QUE O ATS AUMENTOU MAIS DE 1%-----\n')
    for index, row in df[df['Diferenca ATS'] > 1].iterrows():
        log.write('\n')
        linha = row
        log.write(f"{linha}\n")
    log.close()

def matriculas_para_incluir_ats(df, mes, ano):
    log = open('log.txt', 'a+')
    log.write('\n-----SERVIDORES QUE O MENTORH NÃO GEROU ATS-----\n')
    df_SQL = pd.read_csv('.\\ATS - {}-{}\\ats gerados - {:02}-{} SQL.csv'.format(mes, ano, mes, ano), delimiter=';')
    df_SQL['Matricula'] = df_SQL['Matricula'].astype(int)
    df_SQL['Novo_Inicio'] = df_SQL['Novo_Inicio'].apply(lambda x: int(str(x)[-7:-5]) if len(str(x))>1 else mes)
    matriculas_SQL_fora_do_mentorh = df_SQL[df_SQL['Novo_Inicio'] == mes]
    matriculas_SQL_fora_do_mentorh = matriculas_SQL_fora_do_mentorh[matriculas_SQL_fora_do_mentorh['Matricula'].apply(lambda x: x not in list(df['Matricula']))]
    if len(matriculas_SQL_fora_do_mentorh) > 0:
        for index, row in matriculas_SQL_fora_do_mentorh.iterrows():
            log.write('\n')
            linha = row
            log.write(f"{linha}\n")
    log.close()
    return matriculas_SQL_fora_do_mentorh['Matricula']

def matriculas_para_tirar_ats(df, mes, ano):
    log = open('log.txt', 'a+')
    log.write('\n-----SERVIDORES QUE O MENTORH NÃO CONSIDEROU OS AFASTAMENTOS-----\n')
    df_SQL = pd.read_csv('.\\ATS - {}-{}\\ats gerados - {:02}-{} SQL.csv'.format(mes, ano, mes, ano), delimiter=';')
    df_SQL['Matricula'] = df_SQL['Matricula'].astype(int)
    df_SQL['Novo_Inicio'] = df_SQL['Novo_Inicio'].apply(lambda x: int(str(x)[-7:-5]) if len(str(x))>1 else mes)
    matriculas_com_afastamento = df_SQL[df_SQL['Novo_Inicio'] != mes]
    matriculas_com_afastamento = matriculas_com_afastamento[matriculas_com_afastamento['Matricula'].apply(lambda x: x in list(df['Matricula']))]
    if len(matriculas_com_afastamento) > 0:
        for index, row in matriculas_com_afastamento.iterrows():
            log.write('\n')
            linha = row
            log.write(f"{linha}\n")
    log.close()
    return matriculas_com_afastamento['Matricula']


In [53]:
df_ats_mentorh = le_ats_gerados_mentorh(7, 2023)
matriculas_ats = df_ats_mentorh['Matricula']
pulou_mais_de_um(df_ats_mentorh)
matriculas_tirar_ats = matriculas_para_tirar_ats(df_ats_mentorh, 7, 2023)
matriculas_ats = matriculas_ats[matriculas_ats.apply(lambda x: int(x) not in matriculas_tirar_ats.values)]
matriculas_incluir_ats = matriculas_para_incluir_ats(df_ats_mentorh, 7, 2023)
for value in matriculas_incluir_ats.values:
    matriculas_ats.at[len(matriculas_ats)+1] = value

In [54]:
def le_progressoes_geradas_mentorh(mes, ano):
    df = pd.read_csv('.\\PROGRESSAO - {}-{}\\progressoes geradas - {:02}-{}.csv'.format(mes, ano, mes, ano), names=range(8), delimiter=';')
    df_progressoes_mentorh = pd.DataFrame()
    df_progressoes_mentorh['Matricula'] = [int(matricula) for indice, matricula in enumerate(df[0]) if indice%2-1 == 0]
    df_progressoes_mentorh['Nome'] = [nome for indice, nome in enumerate(df[1]) if indice%2-1 == 0]
    df_progressoes_mentorh['Padrao Atual'] = [padrao for indice, padrao in enumerate(df[5]) if indice%2-1 == 0]
    df_progressoes_mentorh['Padrao Novo'] = [padrao for indice, padrao in enumerate(df[6]) if indice%2-1 == 0]
    return df_progressoes_mentorh

def matriculas_para_incluir_progressao(df, mes, ano):
    log = open('log.txt', 'a+')
    log.write('\n-----SERVIDORES QUE O MENTORH NÃO GEROU PROGRESSÃO-----\n')
    df_SQL = pd.read_csv('.\\PROGRESSAO - {}-{}\\progressoes geradas - {:02}-{} SQL.csv'.format(mes, ano, mes, ano), delimiter=';')
    df_SQL['Matricula'] = df_SQL['Matricula'].astype(int)
    fim_da_carreira = ['A69', 'B69', 'C54', 'D39', 'E24']
    matriculas_SQL_fora_do_mentorh = df_SQL[df_SQL['Matricula'].apply(lambda x: x not in list(df['Matricula']))]
    matriculas_SQL_fora_do_mentorh = matriculas_SQL_fora_do_mentorh[list(map(lambda x: x not in fim_da_carreira, matriculas_SQL_fora_do_mentorh['Padrao'].values))]
    if len(matriculas_SQL_fora_do_mentorh) > 0:
        for index, row in matriculas_SQL_fora_do_mentorh.iterrows():
            log.write('\n')
            linha = row
            log.write(f"{linha}\n")
    log.close()
    return matriculas_SQL_fora_do_mentorh['Matricula']

def matriculas_para_tirar_progressao(df, mes, ano):
    log = open('log.txt', 'a+')
    log.write('\n-----SERVIDORES QUE O MENTORH NÃO CONSIDEROU OS AFASTAMENTOS-----\n')
    df_SQL = pd.read_csv('.\\PROGRESSAO - {}-{}\\progressoes geradas - {:02}-{} SQL.csv'.format(mes, ano, mes, ano), delimiter=';')
    df_SQL['Matricula'] = df_SQL['Matricula'].astype(int)
    matriculas_mentorh_fora_do_SQL = df[df['Matricula'].apply(lambda x: x not in list(df_SQL['Matricula']))]
    if len(matriculas_mentorh_fora_do_SQL) > 0:
        for index, row in matriculas_mentorh_fora_do_SQL.iterrows():
            log.write('\n')
            linha = row
            log.write(f"{linha}\n")
    log.close()
    return matriculas_mentorh_fora_do_SQL['Matricula']

In [55]:
df_progressoes_mentorh = le_progressoes_geradas_mentorh(7, 2023)
matriculas_progressao = df_progressoes_mentorh['Matricula']
matriculas_tirar_progressao = matriculas_para_tirar_progressao(df_progressoes_mentorh, 7, 2023)
matriculas_progressao = matriculas_progressao[matriculas_progressao.apply(lambda x: x not in matriculas_tirar_progressao.values)]
matriculas_incluir_progressao = matriculas_para_incluir_progressao(df_progressoes_mentorh, 7, 2023)
for value in matriculas_incluir_progressao.values:
    matriculas_progressao.at[len(matriculas_progressao)+1] = value
matriculas_ats = matriculas_ats[matriculas_ats.apply(lambda x: x not in matriculas_progressao.values)]

In [68]:
def acerta_ferias_ats(matriculas_ats, matriculas_ferias, bases):

    bases_ats = bases[bases['Matricula'].apply(lambda x: x in matriculas_ats.values)]
    bases_ats_ferias = bases_ats[bases_ats['Matricula'].apply(lambda x: x in matriculas_ferias.values)]
    bases_ats_ferias = bases_ats_ferias[['Matricula', 'R1028_Antiga', 'R1028_Nova', 'R1013_Antiga', 'R1014_Antiga']]

    ferias_acertadas = bases_ats_ferias
    ferias_acertadas['R1013_Ajustada'] = np.where(ferias_acertadas['R1013_Antiga'].astype(float) > 0.0, np.trunc((ferias_acertadas['R1028_Nova'] - ferias_acertadas['R1028_Antiga'])*100/3)/100, ferias_acertadas['R1013_Antiga'])
    ferias_acertadas['R1013_Ajustada'] = np.where((ferias_acertadas['R1013_Antiga'].astype(float) + ferias_acertadas['R1013_Ajustada'].astype(float) > TETO/3), np.trunc((TETO/3 - ferias_acertadas['R1013_Antiga'].astype(float))*100)/100, ferias_acertadas['R1013_Ajustada'])
    ferias_acertadas['R1014_Ajustada'] = np.where(ferias_acertadas['R1014_Antiga'].astype(float) > 0.0, np.trunc((ferias_acertadas['R1028_Nova'] - ferias_acertadas['R1028_Antiga'])*133.33/3)/100, ferias_acertadas['R1014_Antiga'])
    ferias_acertadas['R1014_Ajustada'] = np.where((ferias_acertadas['R1014_Antiga'].astype(float) + ferias_acertadas['R1014_Ajustada'].astype(float) > 1.3333*TETO/3), np.trunc((1.3333*TETO/3 - ferias_acertadas['R1014_Antiga'].astype(float))*100)/100, ferias_acertadas['R1014_Ajustada'])

    log = open('log.txt', 'a+')
    log.write('\n-----SERVIDORES COM FERIAS MARCADAS E MUDANCA DE ATS, MAS SEM 1013 E 1014 NA FOLHA NORMAL-----\n')
    for index, row in ferias_acertadas[ferias_acertadas['R1013_Antiga'].astype(float) < 1].iterrows():
        log.write('\n')
        linha = row
        log.write(f"{linha}\n")

    log.write('\n-----SERVIDORES COM FERIAS MARCADAS E MUDANCA DE ATS, MAS JÁ NO TETO DO 1013 E 1014-----\n')
    for index, row in ferias_acertadas[TETO/3 - ferias_acertadas['R1013_Antiga'].astype(float) < 1].iterrows():
        log.write('\n')
        linha = row
        log.write(f"{linha}\n")
    log.close()

    return ferias_acertadas
    

In [64]:
def acerta_ferias_progressao(matriculas_progressao, matriculas_ferias, bases):
    

    bases_progressao = bases[bases['Matricula'].apply(lambda x: x in matriculas_progressao.values)]
    bases_progressao_ferias = bases_progressao[bases_progressao['Matricula'].apply(lambda x: x in matriculas_ferias.values)]
    bases_progressao_ferias = bases_progressao_ferias[['Matricula', 'R1002_Antiga', 'R1007_Antiga', 'R1028_Antiga', 'R1170_Antiga', 'R1013_Antiga', 'R1014_Antiga', 'R1002_Nova', 'R1007_Nova', 'R1028_Nova', 'R1170_Nova']]

    ferias_acertadas = bases_progressao_ferias
    ferias_acertadas['R1013_Ajustada'] = np.where(ferias_acertadas['R1013_Antiga'].astype(float) > 0.0, np.trunc((ferias_acertadas['R1002_Nova'] + ferias_acertadas['R1007_Nova'] + ferias_acertadas['R1028_Nova'] + ferias_acertadas['R1170_Nova'] - ferias_acertadas['R1002_Antiga'] - ferias_acertadas['R1007_Antiga'] - ferias_acertadas['R1028_Antiga'] - ferias_acertadas['R1170_Antiga'])*100/3)/100, ferias_acertadas['R1013_Antiga'])
    ferias_acertadas['R1013_Ajustada'] = np.where((ferias_acertadas['R1013_Antiga'].astype(float) + ferias_acertadas['R1013_Ajustada'].astype(float) > TETO/3), np.trunc((TETO/3 - ferias_acertadas['R1013_Antiga'].astype(float))*100)/100, ferias_acertadas['R1013_Ajustada'])
    ferias_acertadas['R1014_Ajustada'] = np.where(ferias_acertadas['R1014_Antiga'].astype(float) > 0.0, np.trunc((ferias_acertadas['R1002_Nova'] + ferias_acertadas['R1007_Nova'] + ferias_acertadas['R1028_Nova'] + ferias_acertadas['R1170_Nova'] - ferias_acertadas['R1002_Antiga'] - ferias_acertadas['R1007_Antiga'] - ferias_acertadas['R1028_Antiga'] - ferias_acertadas['R1170_Antiga'])*133.33/3)/100, ferias_acertadas['R1014_Antiga'])
    ferias_acertadas['R1014_Ajustada'] = np.where((ferias_acertadas['R1014_Antiga'].astype(float) + ferias_acertadas['R1014_Ajustada'].astype(float) > 1.3333*TETO/3), np.trunc((1.3333*TETO/3 - ferias_acertadas['R1014_Antiga'].astype(float))*100)/100, ferias_acertadas['R1014_Ajustada'])
    
    log = open('log.txt', 'a+')
    log.write('\n-----SERVIDORES COM FERIAS MARCADAS E MUDANCA DE PADRAO, MAS SEM 1013 E 1014 NA FOLHA NORMAL-----\n')
    for index, row in ferias_acertadas[ferias_acertadas['R1013_Antiga'].astype(float) < 1].iterrows():
        log.write('\n')
        linha = row
        log.write(f"{linha}\n")


    log.write('\n-----SERVIDORES COM FERIAS MARCADAS E MUDANCA DE PADRAO, MAS JÁ NO TETO DO 1013 E 1014-----\n')
    for index, row in ferias_acertadas[TETO/3 - ferias_acertadas['R1013_Antiga'].astype(float) < 1].iterrows():
        log.write('\n')
        linha = row
        log.write(f"{linha}\n")

    log.close()

    return ferias_acertadas

In [69]:
mes = 7
ano = 2023

ferias = pd.read_csv('./FERIAS - {}-{}/FERIAS - {}-{}.csv'.format(mes, ano, mes, ano), delimiter=';')
matriculas_ferias = ferias['Matr'][:-2].astype(int)
matriculas_ferias

bases = pd.read_csv('./relatorios/bases_ats_prog {}-{}.csv'.format(mes, ano), delimiter=';')
bases['Matricula'] = bases['Matricula'].astype(int)

ferias_acertadas_ats = acerta_ferias_ats(matriculas_ats, matriculas_ferias, bases)

ferias_acertadas_progressao = acerta_ferias_progressao(matriculas_progressao, matriculas_ferias, bases)



In [80]:
def gera_arquivo_importacao(ferias_acertadas_ats, ferias_acertadas_progressao, mes, ano):
    arq_importacao = open('arq_importacao.txt', 'w')
    for index, row in ferias_acertadas_ats.iterrows():
        linha = str(int(row['Matricula'])) + '\t' + '1013' + '\t' + '0' + '\t' + str(row['R1013_Ajustada']).replace('.', ',')+ '\t' + '1'+ '\t' + '1'+ '\t' + 'V'+ '\t' + 'N'+ '\t' + 'Acerto de férias em decorrência de progressão e/ou aumento de ATS'+ '\t' + '{:02}{}'.format(mes, ano) + '\t' + '0' + '\t' + '0' + '\t' + '{:02}{}'.format(mes, ano)
        arq_importacao.write(f"{linha}\n")
        linha = str(int(row['Matricula'])) + '\t' + '1014' + '\t' + '0' + '\t' + str(row['R1014_Ajustada']).replace('.', ',')+ '\t' + '1'+ '\t' + '1'+ '\t' + 'V'+ '\t' + 'N'+ '\t' + 'Acerto de férias em decorrência de progressão e/ou aumento de ATS'+ '\t' + '{:02}{}'.format(mes, ano) + '\t' + '0' + '\t' + '0' + '\t' + '{:02}{}'.format(mes, ano)
        arq_importacao.write(f"{linha}\n")
    for index, row in ferias_acertadas_progressao.iterrows():
        linha = str(int(row['Matricula'])) + '\t' + '1013' + '\t' + '0' + '\t' + str(row['R1013_Ajustada']).replace('.', ',')+ '\t' + '1'+ '\t' + '1'+ '\t' + 'V'+ '\t' + 'N'+ '\t' + 'Acerto de férias em decorrência de progressão e/ou aumento de ATS'+ '\t' + '{:02}{}'.format(mes, ano) + '\t' + '0' + '\t' + '0' + '\t' + '{:02}{}'.format(mes, ano)
        arq_importacao.write(f"{linha}\n")
        linha = str(int(row['Matricula'])) + '\t' + '1014' + '\t' + '0' + '\t' + str(row['R1014_Ajustada']).replace('.', ',')+ '\t' + '1'+ '\t' + '1'+ '\t' + 'V'+ '\t' + 'N'+ '\t' + 'Acerto de férias em decorrência de progressão e/ou aumento de ATS'+ '\t' + '{:02}{}'.format(mes, ano) + '\t' + '0' + '\t' + '0' + '\t' + '{:02}{}'.format(mes, ano)
        arq_importacao.write(f"{linha}\n")
    arq_importacao.close()

In [81]:
gera_arquivo_importacao(ferias_acertadas_ats, ferias_acertadas_progressao, mes, ano)